In [1]:
# --- 1. SETUP AND DATA LOADING ---

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure settings for all visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Load the dataset from a public URL
# Note: This dataset requires a specific 'latin1' encoding due to special characters.
# This is a common real-world data loading challenge.
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
df = pd.read_excel(url)

print("Dataset loaded successfully!")
df.head()

Dataset loaded successfully!


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
# --- 2. INITIAL DATA INSPECTION ---

# Get a concise summary of the DataFrame (data types, non-null counts)
print("--- DataFrame Info ---")
df.info()

# Check for missing values - this will be a key part of our cleaning process
print("\n--- Missing Values Count ---")
print(df.isnull().sum())

# Generate descriptive statistics to identify anomalies (like negative quantity)
print("\n--- Descriptive Statistics ---")
print(df.describe())

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB

--- Missing Values Count ---
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

--- Descriptive Statistics ---
            Quantity              

### Initial Findings & Data Cleaning Plan

The initial inspection reveals several critical issues that must be addressed before analysis can begin. This will be our data cleaning checklist.

**1. Missing `CustomerID`:**
*   **Problem:** There are 135,080 rows with missing `CustomerID`.
*   **Plan:** Since our analysis will focus on customer behavior, these rows are not useful. We will remove them.

**2. Incorrect Data Type for `CustomerID`:**
*   **Problem:** `CustomerID` is a `float64`, but it should be a whole number (integer) or a string.
*   **Plan:** After handling missing values, we will convert this column to an integer type.

**3. Negative `Quantity`:**
*   **Problem:** The minimum quantity is -80995. These represent cancelled orders or returns.
*   **Plan:** We must filter out these rows, as they do not represent actual sales.

**4. Zero `UnitPrice`:**
*   **Problem:** The minimum unit price is 0. These are not valid transactions.
*   **Plan:** These rows will be removed from the dataset.

**5. `InvoiceDate` as a Timestamp:**
*   **Problem:** The `InvoiceDate` is already a datetime object, which is good.
*   **Plan:** We will need to engineer new features from this column (e.g., month, day of week, hour) to perform time-series analysis.

In [3]:
# --- 3. DATA CLEANING ---

# Create a copy of the dataframe to work on, preserving the original
df_clean = df.copy()

# Print the original shape for comparison
print(f"Original dataset shape: {df_clean.shape}")

# --- Step 1: Remove rows with missing CustomerID ---
# These rows are not useful for customer-level analysis.
df_clean.dropna(subset=['CustomerID'], inplace=True)
print(f"Shape after dropping missing CustomerID: {df_clean.shape}")

# --- Step 2: Change CustomerID to integer type ---
# This must be done AFTER dropping NaNs, as NaN is a float.
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

# --- Step 3: Remove rows with negative Quantity (returns) ---
# We are only interested in sales, not returns.
df_clean = df_clean[df_clean['Quantity'] > 0]
print(f"Shape after removing negative Quantity: {df_clean.shape}")

# --- Step 4: Remove rows with zero UnitPrice ---
# These are not valid commercial transactions.
df_clean = df_clean[df_clean['UnitPrice'] > 0]
print(f"Shape after removing zero UnitPrice: {df_clean.shape}")

print("\nData cleaning complete!")

Original dataset shape: (541909, 8)
Shape after dropping missing CustomerID: (406829, 8)
Shape after removing negative Quantity: (397924, 8)
Shape after removing zero UnitPrice: (397884, 8)

Data cleaning complete!


In [4]:
# --- 4. VERIFYING THE CLEANED DATA ---

# Check the info again to confirm CustomerID type and non-null counts
print("--- Cleaned DataFrame Info ---")
df_clean.info()

# Check for missing values again - CustomerID should now be clean
print("\n--- Missing Values in Cleaned Data ---")
print(df_clean.isnull().sum())

# Run describe() again to confirm that min values for Quantity and UnitPrice are positive
print("\n--- Descriptive Statistics of Cleaned Data ---")
print(df_clean.describe())

--- Cleaned DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  int64         
 7   Country      397884 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 27.3+ MB

--- Missing Values in Cleaned Data ---
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

--- Descriptive Statistics of Cleaned Data ---
            Quantity                    Invoi

### Data Cleaning Summary

The raw dataset has been successfully cleaned according to our established plan. The key actions performed were:

*   **Removed rows with missing `CustomerID`:** This resulted in the removal of 135,080 rows.
*   **Filtered out returns:** An additional 9,288 rows with negative quantities were removed.
*   **Removed invalid transactions:** 40 rows with a unit price of zero were also dropped.
*   **Corrected data type:** The `CustomerID` column was successfully converted to an integer type.

After the cleaning process, our dataset now contains **397,884 valid sales transactions**. It is internally consistent and ready for the next stage: **Feature Engineering**.

In [5]:
# --- 5. FEATURE ENGINEERING ---

# --- Step 1: Create TotalPrice column ---
# This is the most important metric for sales analysis.
df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['UnitPrice']

# --- Step 2: Extract time-based features from InvoiceDate ---
# These new columns will allow us to analyze trends over time (e.g., monthly sales, peak hours).
# We use the .dt accessor because 'InvoiceDate' is a datetime object.

# Extract Month name for easier plotting
df_clean['InvoiceMonth'] = df_clean['InvoiceDate'].dt.strftime('%Y-%m')

# Extract Day of the week name
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.day_name()

# Extract Hour of the day
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour

# --- Step 3: Verify the new columns ---
# Always check your work by displaying the head of the DataFrame.
print("Feature engineering complete. New columns have been added:")
df_clean[['InvoiceDate', 'InvoiceMonth', 'DayOfWeek', 'Hour', 'Quantity', 'UnitPrice', 'TotalPrice']].head()

Feature engineering complete. New columns have been added:


,InvoiceDate,InvoiceMonth,DayOfWeek,Hour,Quantity,UnitPrice,TotalPrice
0,2010-12-01 08:26:00,2010-12,Wednesday,8,6,2.55,15.30
1,2010-12-01 08:26:00,2010-12,Wednesday,8,6,3.39,20.34
2,2010-12-01 08:26:00,2010-12,Wednesday,8,8,2.75,22.00
3,2010-12-01 08:26:00,2010-12,Wednesday,8,6,3.39,20.34
4,2010-12-01 08:26:00,2010-12,Wednesday,8,6,3.39,20.34


### Feature Engineering Summary

To enable a more detailed analysis, we have engineered several new features from the existing data:

1.  **`TotalPrice`**: A new column was created by multiplying `Quantity` and `UnitPrice`. This represents the total value of each transaction line and will be our primary metric for measuring sales performance.

2.  **Time-Based Features**: We extracted the following information from the `InvoiceDate` column:
    *   **`InvoiceMonth`**: The year and month of the transaction (e.g., "2010-12"), which is perfect for analyzing monthly sales trends.
    *   **`DayOfWeek`**: The name of the day (e.g., "Monday"), to identify which days are most popular for shopping.
    *   **`Hour`**: The hour of the transaction (0-23), to find peak shopping times during the day.

With these new features, our dataset is now fully prepared for the main event: **Exploratory Data Analysis (EDA)**. We can now proceed to answer our key business questions.